In [7]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import homm3_battle_server as h3
import threading

import socketserver
import logging
import sys
import json
from datetime import datetime
from typing import Dict

In [30]:
logPath = 'logs'
logsFilename = str(datetime.now().date())
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(f"{logPath}/{logsFilename}.log"),
        logging.StreamHandler()
    ]
)

In [3]:
def singleton(class_):
    instances = {}

    def getinstance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return getinstance

In [4]:
# CONSTANTS
BLOCK_LOGIC = True
DEFAULT_LOGIC = False
COMPUTER = 'LeftUser'
USER = 'RightUser'
TIMEOUT = 0.1

@singleton
class hom3instance:
    def __init__(self):
        # server state variables (default)
        self.current_team = None
        self.tcp_responses_counter = 0
        self.last_connection_timestamp = 0
        self.is_service_active = None

        # game state variables
        self.winner = None
        self.last_team = None

        # computer state
        self.is_computer = None
        self.computer_units = None
        self.computer_possible_moves = None

        # human state
        self.is_player = None
        self.player_units = None
        self.player_possible_moves = None

        # action
        self.action = None

    def get_service_active_state(self) -> bool:
        # show timeout service state response
        current_timestamp = datetime.now().timestamp()
        if (TIMEOUT + self.last_connection_timestamp) - current_timestamp >= 0:
            return True
        else:
            raise Exception('Timeout!! Too slow...')

    def get_winner(self):
        if self.last_team == USER:
            self.winner = USER
        else:
            self.winner = COMPUTER

    def json_handler_logic(self, request):
        logging.info(f'@@@@ {self.current_team} @@@@')
        self.tcp_responses_counter += 1
        self.last_connection_timestamp = datetime.now().timestamp()
        self.is_service_active = self.get_service_active_state()
        # определяем за кого сейчас определяется действие
        if request['currentSide'] == 0:
            self.current_team = USER
            self.is_computer = False
            self.is_player = True
        if request['currentSide'] == 1:
            self.current_team = COMPUTER
            self.is_computer = True
            self.is_player = False

        # if BLOCK_LOGIC:
        #    time.sleep(60)

        logging.info(f'{request}')
        action = {"type": "4"}
        if len(request["actions"]["possibleAttacks"]) > 0:
            attack = request["actions"]["possibleAttacks"][0]
            action = {
                "type": 1 if attack["shooting"] else 2,
                "targetId": attack["defenderId"],
                "moveToHex": attack["moveToHex"]
            }
        elif len(request["actions"]["possibleMoves"]) > 0:
            action = {
                "type": 0,
                "moveToHex": request["actions"]["possibleMoves"][0]
            }
        logging.info(f'{action}')
        logging.info(f'{self.tcp_responses_counter}')
        logging.info(f'@@@@ {self.last_connection_timestamp} @@@@')

        self.action = action
        return action



In [ ]:
class HoMM3BattleTCPHandler(socketserver.BaseRequestHandler):
    def handle(self):
        # когда приходит запрос
        data = self.request.recv(32000).strip()
        # считывается json
        json_data = json.loads(data)
        # TODO: to ML predictions in GYM

        try:
            action = hom3instance().json_handler_logic(request=json_data)
        except Exception as ex:
            raise ex

        self.callback(request = self.request, json_command=action)

    def callback(self, request, json_command):
        request.sendall(json_command.dumps(action).encode('ascii'))


In [8]:
class TcpServer:
    def __init__(self):
        self.server = None

    def start_tcp_server(self, host: str, port: int):
        with socketserver.TCPServer((host, port), HoMM3BattleTCPHandler) as self.server:
            try:
                self.server.allow_reuse_address = True
                self.server.serve_forever(poll_interval=0.1)

            except Exception as ex:
                logging.critical(msg=f'Problem: {str(ex)}\nexit')
                sys.exit(1)

    def stop_tcp_server(self):
        self.server.shutdown()
        self.server.server_close()
        print('Server stopped.')

In [34]:
class HoMM3_B(Env):
    def __init__(self):
        self.state = 5
        pass

    def step(self, action):
        # ждёт запроса от среды о отдаёт действие
        reward = 1
        done = False
        self.state -= 1
        if self.state <= 0:
            done = True
        info = dict()
        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.state = 0
        pass

In [35]:
env = HoMM3_B()

In [36]:
# TODO: TEST AND DEVELOP!
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:1
Episode:2 Score:1
Episode:3 Score:1
Episode:4 Score:1
Episode:5 Score:1
Episode:6 Score:1
Episode:7 Score:1
Episode:8 Score:1
Episode:9 Score:1
Episode:10 Score:1


In [19]:
class HoMM3_Battle_Env(Env):
    def __init__(self):
        # ai service
        self.tcp_server = h3.TcpServer()
        self.tcp_thread = threading.Thread(
            target=self.tcp_server.start_tcp_server,
            args=('localhost', 9999),
            daemon=True
        )
        self.tcp_thread.start()
        # game
        self.vcmi_thread = threading.Thread(
            target=h3.start_vcmi_test_battle,
            args=[False],
            daemon=True
        )
        self.vcmi_thread.start()

        self.action_space = Discrete(4)

        # TODO: update from json
        self.state = 10 + random.randint(-3,3)
        self.stack_length = 10

        # TODO: after init block for step command
        pass

    def step(self, action):
        self.stack_length -= 1
        if self.stack_length <= 0:
            done = True
        else:
            done = False
        self.state += random.randint(-1,1)
        reward = 0
        # placeholder for info
        info = {}
        # return step information
        return self.state, reward, done, info

    def render(self):
        # пока делаем headless
        pass

    def reset(self):
        # self.vcmi_thread.kill()
        # self.tcp_server.kill()
        vcmi_killer_thread = threading.Thread(
            target=h3.kill_vcmi,
            daemon=True
        )
        vcmi_killer_thread.start()
        tcp_killer_thread = threading.Thread(
            target=self.tcp_server.stop_tcp_server,
            daemon=True
        )
        tcp_killer_thread.start()
        vcmi_killer_thread.join()
        tcp_killer_thread.join()
        self.state = 5
        return self.state

In [ ]:
# среда подготавливается и ждёт хода
env = HoMM3_Battle_Env()

In [ ]:
# выключает всё
env.reset()

In [13]:
# TODO: TEST AND DEVELOP!
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Exception in thread Thread-29:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-31:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    Exception in thread Thread-33:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 932, in _bootstrap_inner
self.run()
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 870, in run
Exception in thread Thread-35:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 870, in run
        self._target(*self._args, **self._kwargs)self.run()
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 870, in run
    
self.run

Episode:1 Score:0
Episode:2 Score:0
Episode:3 Score:0
Episode:4 Score:0
Episode:5 Score:0
Episode:6 Score:0
Episode:7 Score:0
Episode:8 Score:0
Episode:9 Score:0
Episode:10 Score:0


AttributeError: 'NoneType' object has no attribute 'shutdown'
        self.run()
  File "/opt/anaconda3/envs/RL_Snake/lib/python3.8/threading.py", line 870, in run
self.server.shutdown()
AttributeError: 'NoneType' object has no attribute 'shutdown'
    self._target(*self._args, **self._kwargs)
  File "/Users/xsa-osx/Documents/GitHub/homm3env/homm3_battle_server.py", line 141, in stop_tcp_server
    self._target(*self._args, **self._kwargs)
  File "/Users/xsa-osx/Documents/GitHub/homm3env/homm3_battle_server.py", line 141, in stop_tcp_server
        self.server.shutdown()
AttributeErrorself.server.shutdown(): 'NoneType' object has no attribute 'shutdown'

AttributeError: 'NoneType' object has no attribute 'shutdown'
